In [1]:
import pandas as pd
import numpy as np
import json

from bz2file import open as bzopen

import pprint

# Get Data from archive using query - 2019

## Iterate over the archive and get related companies and orgs

Повний архів декларацій можна завантажити тут: https://declarations.com.ua/static/data/full_export.json.bz2

**Розділи декларацій, які нас цікавлять:**
  - 7.Цінні папери
  - 8.Корпоративні права
  - 9.Юридичні особи, трасти або інші подібні правові утворення, кінцевим бенефіціарним власником (контролером) яких є суб’єкт декларування або члени його сім’ї
  
  - 15.Робота за сумісництвом суб’єкта декларування
  - 16.Членство суб’єкта декларування в організаціях та їх органах

In [2]:
# Here, we iterate through the full archive and select only filtered declarations from 2019
total_declarations = 0
total_not_2019 = 0
total_corrected = 0
total_not_NACP = 0
total_not_yearly = 0

total_declarations_error_occured = 0
proc_2019_decl_error_occured = 0
total_declarations_final = 0

step_7_securities = 0 
step_8_corp_rights = 0 
step_9_owner_beneficiary = 0 
step_15_part_time_work = 0 
step_16_orgs = 0

result_list_7 = []
result_list_8 = []
result_list_9 = []
result_list_15 = []
result_list_16 = []

        
with bzopen("full_export.json.bz2", "rt", encoding = "utf8") as decl_archive:
    for index, f in enumerate(decl_archive):
        d = json.loads(f) # load one declaration
        
        if 'declaration' in d.keys():
            d = d['declaration']
            
        total_declarations += 1
        error_occured_with_decl = 0

        
        try:
            d_corrected = 1 if (d['related_entities']['documents']['corrected']) else 0
            d_not_NACP = 1 if (d['infocard']['source'] != 'NACP') else 0
            d_not_yearly = 1 if (d['infocard']['document_type'] != 'Щорічна') else 0
            d_not_2019 = 1 if (d['infocard']['declaration_year'] != 2019) else 0

            total_corrected = total_corrected + d_corrected
            total_not_NACP = total_not_NACP + d_not_NACP
            total_not_yearly = total_not_yearly + d_not_yearly
            total_not_2019 = total_not_2019 + d_not_2019
        except:
            error_occured_with_decl = 1
        
        total_declarations_error_occured = total_declarations_error_occured + error_occured_with_decl
        
        if (error_occured_with_decl + d_corrected + d_not_NACP + d_not_yearly + d_not_2019) > 0:
            continue # continue to the next declaration, in case current declaration does not match the requirements
            
        # we reach this point only in case all the filters were met
        total_declarations_final += 1
        
        #if total_declarations_final == 500:
        #    break   
        
        # start the data collection process - get data from some declaration units
        
        try: 
            # some general info about declaration
            current_d_id = d['infocard']['id'], 
            current_d_created_datetime = d['infocard']['created_date']
            current_d_document_type = d['infocard']['document_type'], 
            current_d_year = d['infocard']['declaration_year'],
            current_d_full_name = d['infocard']['last_name'] + ' ' + d['infocard']['first_name'] + ' ' + d['infocard']['patronymic'],
            current_d_office = d['infocard']['office'], 
            current_d_position = d['infocard']['position']


            if ('postType' in d['unified_source']['step_1'].keys()):
                current_d_position_type = d['unified_source']['step_1']['postType']
            else:
                current_d_position_type = np.nan

            if ('postCategory' in d['unified_source']['step_1'].keys()):
                current_d_position_category = d['unified_source']['step_1']['postCategory']
            else:
                current_d_position_category = np.nan

            if ('corruptionAffected' in d['unified_source']['step_1'].keys()):
                current_d_corruptionAffected = d['unified_source']['step_1']['corruptionAffected']
            else:
                current_d_corruptionAffected = np.nan


            if ('dnt_organization_group' in d['unified_source']['step_1'].keys()):
                current_d_dnt_organization_group = d['unified_source']['step_1']['dnt_organization_group']
            else:
                current_d_dnt_organization_group = np.nan

            if ('cityType' in d['unified_source']['step_1'].keys()):
                current_d_cityType = d['unified_source']['step_1']['cityType']
            else:
                current_d_cityType = np.nan

            if ('city' in d['unified_source']['step_1'].keys()):
                current_d_city = d['unified_source']['step_1']['city']
            else:
                current_d_city = np.nan


            # step 7 -- Цінні папери
            if ("step_7" in d['unified_source'].keys()):
                if len(d['unified_source']['step_7']) > 0: # check if there is data in current step
                    step_7_securities += 1
                    for c in d['unified_source']['step_7'].keys():
                        company_ua = np.nan
                        company_id = ''
                        company_name = ''
                        person = ''
                        persons= ''

                        if ('emitent_ua_company_code' in d['unified_source']['step_7'][c].keys()):
                            if (d['unified_source']['step_7'][c]['emitent_ua_company_code'] != ''):
                                    company_ua = 1
                                    company_id = d['unified_source']['step_7'][c]['emitent_ua_company_code']
                                    company_name = d['unified_source']['step_7'][c]['emitent_ua_company_name']
                        if company_id == '':
                            if ('emitent_eng_company_code' in d['unified_source']['step_7'][c].keys()):
                                if (d['unified_source']['step_7'][c]['emitent_eng_company_code'] != ''):
                                    company_ua = 0
                                    company_id = d['unified_source']['step_7'][c]['emitent_eng_company_code']
                                    company_name = d['unified_source']['step_7'][c]['emitent_eng_company_name']

                        if ('person' in d['unified_source']['step_7'][c].keys()):
                            person = d['unified_source']['step_7'][c]['person']

                        if ('persons' in d['unified_source']['step_7'][c].keys()):
                            persons = d['unified_source']['step_7'][c]['persons']

                        result_list_7.append(pd.DataFrame([{'id': current_d_id[0],
                                                            'created_datetime': current_d_created_datetime,
                                                            'index': index,
                                                            'document_type': current_d_document_type[0],
                                                            'year': current_d_year[0],
                                                            'full_name': current_d_full_name[0],
                                                            'office': current_d_office[0],
                                                            'position': current_d_position,
                                                            'position_type': current_d_position_type,
                                                            'position_category': current_d_position_category,
                                                            'corruptionAffected': current_d_corruptionAffected,
                                                            'dnt_organization_group' : current_d_dnt_organization_group,
                                                            'city' : current_d_city,
                                                            'city_type' : current_d_cityType,
                                                            'relation_type': 7, 
                                                            'person': person,      
                                                            'persons': persons,
                                                            'company_ua': company_ua,
                                                            'company_id': company_id, 
                                                            'company_name': company_name}]))


            # step 8 -- Корпоративні права
            if ("step_8" in d['unified_source'].keys()):
                if len(d['unified_source']['step_8']) > 0: # check if there is data in current step
                    step_8_corp_rights += 1
                    for c in d['unified_source']['step_8'].keys():
                        company_ua = np.nan
                        company_id = ''
                        company_name = ''
                        person = ''
                        persons= ''

                        if d['unified_source']['step_8'][c]['country'] == '1':
                            company_ua = 1
                        else:
                            company_ua = 0

                        if ('corporate_rights_company_code' in d['unified_source']['step_8'][c].keys()):
                            company_id = d['unified_source']['step_8'][c]['corporate_rights_company_code']

                        if ('name' in d['unified_source']['step_8'][c].keys()):
                            if (d['unified_source']['step_8'][c]['name'] != ''):
                                company_name = d['unified_source']['step_8'][c]['name']
                        if company_name == '':
                            if ('en_name' in d['unified_source']['step_8'][c].keys()):
                                if (d['unified_source']['step_8'][c]['en_name'] != ''):
                                    company_name = d['unified_source']['step_8'][c]['en_name']

                        if ('person' in d['unified_source']['step_8'][c].keys()):
                            person = d['unified_source']['step_8'][c]['person']

                        if ('persons' in d['unified_source']['step_8'][c].keys()):
                            persons = d['unified_source']['step_8'][c]['persons']

                        result_list_8.append(pd.DataFrame([{'id': current_d_id[0],
                                                            'created_datetime': current_d_created_datetime,                                                    
                                                            'index': index,
                                                            'document_type': current_d_document_type[0],
                                                            'year': current_d_year[0],
                                                            'full_name': current_d_full_name[0],
                                                            'office': current_d_office[0],
                                                            'position': current_d_position,
                                                            'position_type': current_d_position_type,
                                                            'position_category': current_d_position_category,
                                                            'corruptionAffected': current_d_corruptionAffected,
                                                            'dnt_organization_group' : current_d_dnt_organization_group,
                                                            'city' : current_d_city,
                                                            'city_type' : current_d_cityType,
                                                            'relation_type': 8, 
                                                            'person': person,      
                                                            'persons': persons,
                                                            'company_ua': company_ua,
                                                            'company_id': company_id, 
                                                            'company_name': company_name}]))



            # step 9 -- Кінцевий бенефіціарний власник (контролер)
            if ("step_9" in d['unified_source'].keys()):
                if len(d['unified_source']['step_9']) > 0: # check if there is data in current step
                    step_9_owner_beneficiary += 1
                    for c in d['unified_source']['step_9'].keys():
                        company_ua = np.nan
                        company_id = ''
                        company_name = ''
                        person = ''
                        persons= ''


                        if ('country_beneficial_owner' in d['unified_source']['step_9'][c].keys()):
                            if (d['unified_source']['step_9'][c]['country_beneficial_owner'] != '1'):
                                company_ua = 0
                            else:
                                company_ua = 1


                        if ('ua_company_code_beneficial_owner' in d['unified_source']['step_9'][c].keys()):
                            if (d['unified_source']['step_9'][c]['ua_company_code_beneficial_owner'] != ''):
                                    company_id = d['unified_source']['step_9'][c]['ua_company_code_beneficial_owner']
                                    company_ua = 1
                        if company_id == '':
                            if ('company_code_beneficial_owner' in d['unified_source']['step_9'][c].keys()):
                                if (d['unified_source']['step_9'][c]['company_code_beneficial_owner'] != ''):
                                    company_id = d['unified_source']['step_9'][c]['company_code_beneficial_owner']
                                    company_ua = 0

                        if ('company_name_beneficial_owner' in d['unified_source']['step_9'][c].keys()):
                            if (d['unified_source']['step_9'][c]['company_name_beneficial_owner'] != ''):
                                company_name = d['unified_source']['step_9'][c]['company_name_beneficial_owner']
                        if company_name == '':
                            if ('en_company_name_beneficial_owner' in d['unified_source']['step_9'][c].keys()):
                                if (d['unified_source']['step_9'][c]['en_company_name_beneficial_owner'] != ''):
                                    company_name = d['unified_source']['step_9'][c]['en_company_name_beneficial_owner']

                        if ('person' in d['unified_source']['step_9'][c].keys()):
                            try:
                                person_keys = d['unified_source']['step_9'][c]['person'].keys()
                                if '1' in person_keys:
                                    person = 1
                                else:
                                    person = list(person_keys)[0]
                            except:
                                print('person does not have keys')
                                person = d['unified_source']['step_9'][c]['person']


                        if ('persons' in d['unified_source']['step_9'][c].keys()):
                            persons = d['unified_source']['step_9'][c]['persons']


                        result_list_9.append(pd.DataFrame([{'id': current_d_id[0],
                                                            'created_datetime': current_d_created_datetime,
                                                            'index': index,
                                                            'document_type': current_d_document_type[0],
                                                            'year': current_d_year[0],
                                                            'full_name': current_d_full_name[0],
                                                            'office': current_d_office[0],
                                                            'position': current_d_position,
                                                            'position_type': current_d_position_type,
                                                            'position_category': current_d_position_category,
                                                            'corruptionAffected': current_d_corruptionAffected,
                                                            'dnt_organization_group' : current_d_dnt_organization_group,
                                                            'city' : current_d_city,
                                                            'city_type' : current_d_cityType,
                                                            'relation_type': 9, 
                                                            'person': person,      
                                                            'persons': persons,
                                                            'company_ua': company_ua,
                                                            'company_id': company_id, 
                                                            'company_name': company_name}]))


            # step 15 -- Робота за сумісництвом суб’єкта декларування
            if ("step_15" in d['unified_source'].keys()):
                if len(d['unified_source']['step_15']) > 0: # check if there is data in current step
                    step_15_part_time_work += 1
                    for c in d['unified_source']['step_15'].keys():
                        company_ua = np.nan
                        company_id = ''
                        company_name = ''
                        person = ''
                        persons = ''
                        description = ''
                        emitent_citizen = ''
                        paid = ''

                        if ('emitent_ua_company_code' in d['unified_source']['step_15'][c].keys()):
                            if (d['unified_source']['step_15'][c]['emitent_ua_company_code'] != ''):
                                    company_ua = 1
                                    company_id = d['unified_source']['step_15'][c]['emitent_ua_company_code']
                                    company_name = d['unified_source']['step_15'][c]['emitent_ua_company_name']
                        if company_id == '':
                            if ('emitent_eng_company_code' in d['unified_source']['step_15'][c].keys()):
                                if (d['unified_source']['step_15'][c]['emitent_eng_company_code'] != ''):
                                    company_ua = 0
                                    company_id = d['unified_source']['step_15'][c]['emitent_eng_company_code']
                                    company_name = d['unified_source']['step_15'][c]['emitent_eng_company_name']


                        if ('description' in d['unified_source']['step_15'][c].keys()):
                            description = d['unified_source']['step_15'][c]['description']

                        if ('emitent_citizen' in d['unified_source']['step_15'][c].keys()):
                            emitent_citizen = d['unified_source']['step_15'][c]['emitent_citizen']

                        if ('paid' in d['unified_source']['step_15'][c].keys()):
                            paid = d['unified_source']['step_15'][c]['paid']

                        result_list_15.append(pd.DataFrame([{'id': current_d_id[0],
                                                            'created_datetime': current_d_created_datetime,
                                                            'index': index,
                                                            'document_type': current_d_document_type[0],
                                                            'year': current_d_year[0],
                                                            'full_name': current_d_full_name[0],
                                                            'office': current_d_office[0],
                                                            'position': current_d_position,
                                                            'position_type': current_d_position_type,
                                                            'position_category': current_d_position_category,
                                                            'corruptionAffected': current_d_corruptionAffected,
                                                            'dnt_organization_group' : current_d_dnt_organization_group,
                                                            'city' : current_d_city,
                                                            'city_type' : current_d_cityType,
                                                            'relation_type': 15, 
                                                            'person': person,      
                                                            'persons': persons,
                                                            'company_ua': company_ua,
                                                            'company_id': company_id, 
                                                            'company_name': company_name,
                                                            'description': description, 
                                                            'emitent_citizen': emitent_citizen,
                                                            'paid': paid}]))


            # step 16 -- Членство суб’єкта декларування в організаціях та їх органах
            if ("step_16" in d['unified_source'].keys()):
                # for orgs
                if len(d['unified_source']['step_16']['org']) > 0: # check if there is data in current step
                    step_16_orgs += 1
                    for c in d['unified_source']['step_16']['org'].keys():
                        org_name = ''
                        org_type = ''
                        org_id = ''
                        org_unit_name = 'org'
                        org_unit_type = 'org'

                        if ('objectName' in d['unified_source']['step_16']['org'][c].keys()):
                            org_name = d['unified_source']['step_16']['org'][c]['objectName']

                        if ('objectType' in d['unified_source']['step_16']['org'][c].keys()):
                            org_type = d['unified_source']['step_16']['org'][c]['objectType']

                        if ('reestrCode' in d['unified_source']['step_16']['org'][c].keys()):
                            org_id = d['unified_source']['step_16']['org'][c]['reestrCode']

                        result_list_16.append(pd.DataFrame([{'id': current_d_id[0],
                                                            'created_datetime': current_d_created_datetime,
                                                            'index': index,
                                                            'document_type': current_d_document_type[0],
                                                            'year': current_d_year[0],
                                                            'full_name': current_d_full_name[0],
                                                            'office': current_d_office[0],
                                                            'position': current_d_position,
                                                            'position_type': current_d_position_type,
                                                            'position_category': current_d_position_category,
                                                            'corruptionAffected': current_d_corruptionAffected,
                                                            'dnt_organization_group' : current_d_dnt_organization_group,
                                                            'city' : current_d_city,
                                                            'city_type' : current_d_cityType,
                                                            'relation_type': 16, 
                                                            'org_name': org_name,
                                                            'org_type': org_type,
                                                            'org_id': org_id,
                                                            'org_unit_name': org_unit_name,
                                                            'org_unit_type': org_unit_type}]))

                # for org parts
                if len(d['unified_source']['step_16']['part_org']) > 0: # check if there is data in current step
                    for c in d['unified_source']['step_16']['part_org'].keys():
                        org_name = ''
                        org_type = ''
                        org_id = ''
                        org_unit_name = ''
                        org_unit_type = ''

                        if ('objectName' in d['unified_source']['step_16']['part_org'][c].keys()):
                            org_name = d['unified_source']['step_16']['part_org'][c]['objectName']

                        if ('objectType' in d['unified_source']['step_16']['part_org'][c].keys()):
                            org_type = d['unified_source']['step_16']['part_org'][c]['objectType']

                        if ('reestrCode' in d['unified_source']['step_16']['part_org'][c].keys()):
                            org_id = d['unified_source']['step_16']['part_org'][c]['reestrCode']  

                        if ('unitName' in d['unified_source']['step_16']['part_org'][c].keys()):
                            org_unit_name = d['unified_source']['step_16']['part_org'][c]['unitName']   

                        if ('unitType' in d['unified_source']['step_16']['part_org'][c].keys()):
                            org_unit_type = d['unified_source']['step_16']['part_org'][c]['unitType']

                        result_list_16.append(pd.DataFrame([{'id': current_d_id[0],
                                                            'created_datetime': current_d_created_datetime,
                                                            'index': index,
                                                            'document_type': current_d_document_type[0],
                                                            'year': current_d_year[0],
                                                            'full_name': current_d_full_name[0],
                                                            'office': current_d_office[0],
                                                            'position': current_d_position,
                                                            'position_type': current_d_position_type,
                                                            'position_category': current_d_position_category,
                                                            'corruptionAffected': current_d_corruptionAffected,
                                                            'dnt_organization_group' : current_d_dnt_organization_group,
                                                            'city' : current_d_city,
                                                            'city_type' : current_d_cityType,
                                                            'relation_type': 16, 
                                                            'org_name': org_name,
                                                            'org_type': org_type,
                                                            'org_id': org_id,
                                                            'org_unit_name': org_unit_name,
                                                            'org_unit_type': org_unit_type}]))
        except:
            proc_2019_decl_error_occured += 1

person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does not have keys
person does 

In [3]:
print("Not 2019: ", total_not_2019, 
      "\nNot Yearly: ", total_not_yearly, 
      "\nNot NACP: ", total_not_NACP, 
      "\nCorrected: ", total_corrected)

print("\nTotal processed declarations: ", total_declarations,
      "\nRemained declarations: ", total_declarations_final,
      "\nError occured with declarations: ", total_declarations_error_occured,
      "\nError occured with 2019 dececlaration: ", proc_2019_decl_error_occured)
        
print(
    "\n7.Цінні папери: ", step_7_securities, 
    "\n8.Корпоративні права: ", step_8_corp_rights,
    "\n9.Кінцевий бенефіціарний власник (контролер): ", step_9_owner_beneficiary,
    "\n15.Робота за сумісництвом суб’єкта декларування: ", step_15_part_time_work,
    "\n16.Членство суб’єкта декларування в організаціях та їх органах: ", step_16_orgs
)

Not 2019:  4182789 
Not Yearly:  1373440 
Not NACP:  28788 
Corrected:  509763

Total processed declarations:  5449382 
Remained declarations:  854986 
Error occured with declarations:  0 
Error occured with 2019 dececlaration:  18

7.Цінні папери:  15512 
8.Корпоративні права:  18189 
9.Кінцевий бенефіціарний власник (контролер):  14074 
15.Робота за сумісництвом суб’єкта декларування:  26000 
16.Членство суб’єкта декларування в організаціях та їх органах:  74194


In [4]:
print('Кількість пар декрація-команія\організація за розділами: ')
print("7.Цінні папери: ", len(result_list_7),
       "\n8.Корпоративні права: ", len(result_list_8),
      "\n9.Кінцевий бенефіціарний власник (контролер): ", len(result_list_9),
      "\n15.Робота за сумісництвом суб’єкта декларування: ", len(result_list_15),
      "\n16.Членство суб’єкта декларування в організаціях та їх органах: ", len(result_list_16),
)

Кількість пар декрація-команія\організація за розділами: 
7.Цінні папери:  22263 
8.Корпоративні права:  31951 
9.Кінцевий бенефіціарний власник (контролер):  25782 
15.Робота за сумісництвом суб’єкта декларування:  32392 
16.Членство суб’єкта декларування в організаціях та їх органах:  106649


## Create summary dataframe for all companies (from steps 7,8,9,15)

In [5]:
companies_df = pd.DataFrame({
    'id': [], 
    'created_datetime': [],
    'index': [], 
    'document_type': [], 
    'year': [], 
    'full_name': [], 
    'office': [], 
    'position': [],
    'position_type': [],
    'position_category': [],
    'corruptionAffected': [],
    'dnt_organization_group' : [],
    'city' : [],
    'city_type' : [],
    'relation_type': [], 
    'person': [], 
    'persons': [], 
    'company_ua': [],
    'company_id': [], 
    'company_name': [],
    'description': [],
    'emitent_citizen': [],
    'paid': []
})                           

In [6]:
companies_df = companies_df.append(result_list_7)
companies_df = companies_df.append(result_list_8)
companies_df = companies_df.append(result_list_9)
companies_df = companies_df.append(result_list_15)

In [7]:
companies_df.head(3)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,...,city_type,relation_type,person,persons,company_ua,company_id,company_name,description,emitent_citizen,paid
0,nacp_756fc5a8-2bc8-4e32-8e39-2618cf363267,2020-03-04T14:26:27,105.0,Щорічна,2019.0,Маренчук Олена Григорівна,Фінансове управління Первомайської райдержадмі...,начальник управління,Посада державної служби,Б,...,Первомайськ / Миколаївська область / Україна,7.0,,,1.0,00210996,"АТВТ ""Первомайськдизельмаш""",NaN,NaN,NaN
0,nacp_756fc5a8-2bc8-4e32-8e39-2618cf363267,2020-03-04T14:26:27,105.0,Щорічна,2019.0,Маренчук Олена Григорівна,Фінансове управління Первомайської райдержадмі...,начальник управління,Посада державної служби,Б,...,Первомайськ / Миколаївська область / Україна,7.0,,,1.0,00210996,"АТВТ ""Первомайськдизельмаш""",NaN,NaN,NaN
0,nacp_6ccc466f-03e2-42c5-97b3-9af9032c5285,2020-03-10T08:55:15,134.0,Щорічна,2019.0,Ткаченко Олена Миколаївна,Білгород-Дністровський міськрайонний суд Одесь...,Секретар судового засідання,Посада державної служби,В,...,Білгород-Дністровський / Одеська область / Укр...,7.0,,,NaN,,,NaN,NaN,NaN


In [58]:
companies_df.groupby('relation_type')[['id']].nunique()

,id
relation_type,
7.0,14593
8.0,13552
9.0,9482
15.0,24003


## Create summary dataframe for all organizations (from step 16)

In [8]:
organizations_df = pd.DataFrame({
    'id': [],
    'created_datetime': [],
    'index': [], 
    'document_type': [], 
    'year': [], 
    'full_name': [], 
    'office': [], 
    'position': [],
    'position_type': [],
    'position_category': [],
    'corruptionAffected': [],
    'dnt_organization_group' : [],
    'city' : [],
    'city_type' : [],
    'relation_type': [], 
    'org_name': [], 
    'org_type': [], 
    'org_id': [],
    'org_unit_name': [], 
    'org_unit_type': []
})   

In [9]:
organizations_df = organizations_df.append(result_list_16)
organizations_df.head(3)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,corruptionAffected,dnt_organization_group,city,city_type,relation_type,org_name,org_type,org_id,org_unit_name,org_unit_type
0,nacp_6d38f3cd-07da-43ae-b9bc-d0dd5411f26f,2020-03-03T15:14:57,25.0,Щорічна,2019.0,Ількович Богдан Миколайович,Управління Державної служби України з надзвича...,Начальник служби охорони праці,,,Ні,"Кабмін, міністерства та підлеглі органи",1.2.21.2.48.8.63.1.1,Сторожниця / Ужгородський район / Закарпатська...,16.0,"Фізкультурно-спортивне товариство ""Динамо""",Громадське об’єднання,02942290,org,org
0,nacp_756ff0a4-0983-4e48-95d2-13927de77b19,2020-03-26T14:20:53,117.0,Щорічна,2019.0,Пецур Володимир Володимирович,Тернопільський відділ поліції ГУНП в Тернопіль...,старший дільничний офіцер поліції СДОП ВППП Те...,,,Ні,"Кабмін, міністерства та підлеглі органи",,Тернопіль / Тернопільська область / Україна,16.0,Тернопільска обласна профспілкова організація ...,Саморегулівне чи самоврядне професійне об’єднання,26197081,org,org
0,nacp_68502dc0-7569-4b41-ba82-181da19ac9fb,2020-03-05T09:38:48,120.0,Щорічна,2019.0,Іванов Олег Миколайович,"ЧФ ДП АМПУ, Адміністрація морських портів України",заступник начальника Служби виробничої безпеки,Посадова особа юридичної особи публічного права,,Ні,"Кабмін, міністерства та підлеглі органи",,Чорноморськ / Одеська область / Україна,16.0,ПП МПЧ,Саморегулівне чи самоврядне професійне об’єднання,38549392,org,org


In [59]:
organizations_df.groupby('relation_type')[['id']].nunique()

,id
relation_type,
16.0,72619


## Save results

In [73]:
unique_declarations_with_connection = set(list(organizations_df.id) + list(companies_df.id))
print(len(unique_declarations_with_connection))

unique_declarations_with_connection = set(list(organizations_df.full_name_office_position) + list(companies_df.full_name_office_position))
print(len(unique_declarations_with_connection))

print(len(organizations_df), len(companies_df))

119097
119023
93699 80203


In [69]:
unique_company = set(list(companies_df.id))
unique_org = set(list(organizations_df.id))
unique_both = set(list(unique_company & unique_org))

print(len(unique_company)-len(unique_both), (len(unique_company)-len(unique_both))/len(unique_declarations_with_connection))
print(len(unique_org)-len(unique_both), (len(unique_org)-len(unique_both))/len(unique_declarations_with_connection))
print(len(unique_both), len(unique_both)/len(unique_declarations_with_connection))

print(len(set(list(unique_org) + list(unique_company))))


46478 0.3902533229216521
64216 0.5391907436795217
8403 0.07055593339882617
119097


In [67]:
54881+72619-8403

119097

In [10]:
# convert datetime data to datetime format
organizations_df['created_datetime'] = pd.to_datetime(organizations_df['created_datetime'])
companies_df['created_datetime'] = pd.to_datetime(companies_df['created_datetime'])

In [11]:
print("Companies and organizations initial data sets created.")

Companies and organizations initial data sets created.


In [12]:
companies_df.to_csv('company_01_initial_df.csv', index=False)
organizations_df.to_csv('org_01_initial_df.csv', index=False)

## Process data - verify declaration uniqueness, create drop lists, check company codes, create complanie dict and save the data sets

### Remove declarations with diferent ids but from the same people

In [13]:
companies_df['full_name_office_position'] = companies_df.full_name  + ' ' + companies_df.office + ' ' + companies_df.position
companies_df['full_name_office_position'] = companies_df['full_name_office_position'].str.lower()


organizations_df['full_name_office_position'] = organizations_df.full_name  + ' ' + organizations_df.office + ' ' + organizations_df.position
organizations_df['full_name_office_position'] = organizations_df['full_name_office_position'].str.lower()


companies_df['full_name_office_position']

0    маренчук олена григорівна фінансове управління...
0    маренчук олена григорівна фінансове управління...
0    ткаченко олена миколаївна білгород-дністровськ...
0    передненко олександр григорович комунальне під...
0    передненко олександр григорович комунальне під...
                           ...                        
0    рудович андрій анатолійович вінницька обласна ...
0    ситников олександр олександрович прат очаківсь...
0    радченко вячеслав анатолійович голосіївський р...
0    корецький віктор кирилович заболотттівська сел...
0    іллар леонард амброзійович ратівецька зош і-іі...
Name: full_name_office_position, Length: 112388, dtype: object

In [14]:
comp_people_map = companies_df.groupby(["full_name_office_position"])["id"].nunique().reset_index(name="unique_guids")
print('companies')
print(len(comp_people_map[comp_people_map.unique_guids > 1]))

print('organizations')
org_people_map = organizations_df.groupby(["full_name_office_position"])["id"].nunique().reset_index(name="unique_guids")
print(len(org_people_map[org_people_map.unique_guids > 1]))


companies
3029
organizations
2809


In [15]:
print('companies')
print(len(companies_df.full_name), len(set(companies_df.full_name)))
print(len(companies_df.id), len(set(companies_df.id)))
print(len(comp_people_map.full_name_office_position), len(set(comp_people_map.full_name_office_position)))

print('organizations')
print(len(organizations_df.full_name), len(set(organizations_df.full_name)))
print(len(organizations_df.id), len(set(organizations_df.id)))
print(len(org_people_map.full_name_office_position), len(set(org_people_map.full_name_office_position)))

companies
112388 54181
112388 58122
54842 54842
organizations
106649 71451
106649 75608
72594 72594


In [16]:
companies_df = companies_df.merge(comp_people_map, how='left', on='full_name_office_position')
companies_df.head(1)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,...,person,persons,company_ua,company_id,company_name,description,emitent_citizen,paid,full_name_office_position,unique_guids
0,nacp_756fc5a8-2bc8-4e32-8e39-2618cf363267,2020-03-04 14:26:27,105.0,Щорічна,2019.0,Маренчук Олена Григорівна,Фінансове управління Первомайської райдержадмі...,начальник управління,Посада державної служби,Б,...,,,1.0,00210996,"АТВТ ""Первомайськдизельмаш""",NaN,NaN,NaN,маренчук олена григорівна фінансове управління...,1


In [17]:
unique_naming_companies_df = companies_df.sort_values('created_datetime').drop_duplicates(['full_name_office_position', 
                                                                                           'company_id'], 
                                                                                          keep='last')
print('companies_df')
print(len(companies_df.full_name), len(set(companies_df.full_name)))
print(len(companies_df.id), len(set(companies_df.id)))

print('unique_naming_companies_df')
print(len(unique_naming_companies_df.full_name), len(set(unique_naming_companies_df.full_name)))
print(len(unique_naming_companies_df.id), len(set(unique_naming_companies_df.id)))


companies_df
112388 54181
112388 58122
unique_naming_companies_df
80203 54181
80203 54881


In [18]:
organizations_df = organizations_df.merge(org_people_map, how='left', on='full_name_office_position')
organizations_df.head(1)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,...,city,city_type,relation_type,org_name,org_type,org_id,org_unit_name,org_unit_type,full_name_office_position,unique_guids
0,nacp_6d38f3cd-07da-43ae-b9bc-d0dd5411f26f,2020-03-03 15:14:57,25.0,Щорічна,2019.0,Ількович Богдан Миколайович,Управління Державної служби України з надзвича...,Начальник служби охорони праці,,,...,1.2.21.2.48.8.63.1.1,Сторожниця / Ужгородський район / Закарпатська...,16.0,"Фізкультурно-спортивне товариство ""Динамо""",Громадське об’єднання,02942290,org,org,ількович богдан миколайович управління державн...,1


In [19]:
unique_naming_organizations_df = organizations_df.sort_values('created_datetime').drop_duplicates(['full_name_office_position', 
                                                                                           'org_id'], 
                                                                                          keep='last')
print('organizations_df')
print(len(organizations_df.full_name), len(set(organizations_df.full_name)))
print(len(organizations_df.id), len(set(organizations_df.id)))

print('unique_naming_organizations_df')
print(len(unique_naming_organizations_df.full_name), len(set(unique_naming_organizations_df.full_name)))
print(len(unique_naming_organizations_df.id), len(set(unique_naming_organizations_df.id)))


organizations_df
106649 71451
106649 75608
unique_naming_organizations_df
93699 71451
93699 72619


In [20]:
#Save files to keep track

#unique_naming_companies_df.to_csv('no_dupl_companies_df.csv', index=False)
#companies_df.to_csv('with_dupl_companies_df.csv', index=False)

#unique_naming_organizations_df.to_csv('no_dupl_organizations_df.csv', index=False)
#organizations_df.to_csv('with_dupl_organizations_df.csv', index=False)

In [21]:
companies_df = unique_naming_companies_df
organizations_df = unique_naming_organizations_df

### Create drop lists -- lists of companies that appear just for a single time 
to use it later for filtering out them from the graphs.

See how many folks have a single related company

In [22]:
by_decl_id = companies_df.groupby(["id"])["company_id"].count().reset_index(name="count_companies")
by_decl_id.head(5)

,id,count_companies
0,nacp_00005a2b-c890-432d-a85e-14332cf6abad,1
1,nacp_000246d6-7f39-4fd7-bbc8-1f2a86016f02,2
2,nacp_00026090-0bf5-4566-b70d-d51641f9dbd4,1
3,nacp_0003da2f-23c4-4c91-b261-fc1b208ff0ca,1
4,nacp_00053f03-84f7-46ae-bb5e-501e60e202d6,1


In [23]:
by_decl_id.groupby(["count_companies"])["id"].count().reset_index(name="count_users")

,count_companies,count_users
0,1,42679
1,2,7562
2,3,2286
3,4,977
4,5,482
5,6,278
6,7,168
7,8,105
8,9,73
9,10,56


There are 115 users who have just a single company. Create a list of the for filter them out later on.

In [24]:
by_decl_id.loc[by_decl_id['count_companies'] == 1]

,id,count_companies
0,nacp_00005a2b-c890-432d-a85e-14332cf6abad,1
2,nacp_00026090-0bf5-4566-b70d-d51641f9dbd4,1
3,nacp_0003da2f-23c4-4c91-b261-fc1b208ff0ca,1
4,nacp_00053f03-84f7-46ae-bb5e-501e60e202d6,1
5,nacp_0005678b-7e47-4b0c-9f68-17c5b1f41db2,1
...,...,...
54872,nacp_fff776f0-e2fd-4fa5-8bf1-4435c0bbff14,1
54874,nacp_fffb7b17-6874-4fb1-8a5e-f38e9dfa6324,1
54876,nacp_fffdbb82-1bc1-4b14-a223-3c5487ff95e8,1
54877,nacp_fffe5b71-13c5-42ba-8e4f-77a31f9f42a4,1


In [25]:
by_company_id = companies_df.groupby(["company_id"])["id"].count().reset_index(name="count_decl")
by_company_id.loc[by_company_id['count_decl'] == 1]

,company_id,count_decl
4,0000000000,1
5,00000000000,1
6,00000001,1
7,000001301,1
8,000001353,1
...,...,...
47568,SL26907,1
47569,SL31076,1
47570,SO306550,1
47571,VGG322291094,1


In [26]:
folks_with_single_company = list(by_decl_id.loc[by_decl_id['count_companies'] == 1]['id'])
companies_appear_single_time = list(by_company_id.loc[by_company_id['count_decl'] == 1]['company_id'])

### Check the organization codes and correct them

In [27]:
organizations_df.head(2)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,...,city,city_type,relation_type,org_name,org_type,org_id,org_unit_name,org_unit_type,full_name_office_position,unique_guids
8374,nacp_c1b16f35-07e3-4066-a24e-9d071b80c9c3,2019-03-02 20:07:51,504244.0,Щорічна,2019.0,Чучман Леся Миколаївна,Золочівська міська рада Львівської області,депутат Золочівська міська ради Львівської обл...,,,...,1.2.46.2.18.1.1,Золочів / Золочівський район / Львівська облас...,16.0,"Орган самоорганізації вуличного комітету ""Джер...",Громадське об’єднання,37780812,org,org,чучман леся миколаївна золочівська міська рада...,1
8375,nacp_c1b16f35-07e3-4066-a24e-9d071b80c9c3,2019-03-02 20:07:51,504244.0,Щорічна,2019.0,Чучман Леся Миколаївна,Золочівська міська рада Львівської області,депутат Золочівська міська ради Львівської обл...,,,...,1.2.46.2.18.1.1,Золочів / Золочівський район / Львівська облас...,16.0,"Орган самоорганізації вуличного комітету ""Джер...",Громадське об’єднання,3778812,Вуличний комітет,Керівні органи,чучман леся миколаївна золочівська міська рада...,1


In [28]:
# check if we have empty company IDs or names
print('total records: ', len(organizations_df))

empty_ids = organizations_df.loc[(organizations_df['org_id'] == '')] 
print('\nrecords with empty ids: ', len(empty_ids))
print('empty ids share: ', len(empty_ids)/len(organizations_df))

empty_names = organizations_df.loc[(organizations_df['org_name'] == '')] 
print('\nrecords with empty names: ', len(empty_names))
print('empty names share: ', len(empty_names)/len(organizations_df))


empty_names_ids = organizations_df.loc[((organizations_df['org_name'] == '')) & (organizations_df['org_id'] == '')] 
print('\nrecords with both empty names and empty IDs: ', len(empty_names_ids))
print('both empty names and empty IDs share: ', len(empty_names_ids)/len(organizations_df))


total records:  93699

records with empty ids:  0
empty ids share:  0.0

records with empty names:  0
empty names share:  0.0

records with both empty names and empty IDs:  0
both empty names and empty IDs share:  0.0


In [29]:
clean_organizations_df = organizations_df.loc[organizations_df['org_id'] != ''].copy(deep=True)
len(clean_organizations_df)

93699

In [30]:
print('orgs with correct EDRPOU length: ', 
      len(clean_organizations_df[(clean_organizations_df['org_id'].str.len() == 8)]))

print('orgs with non-8-char EDRPOU length: ', 
      len(clean_organizations_df[(clean_organizations_df['org_id'].str.len() != 8)]))

orgs with correct EDRPOU length:  93311
orgs with non-8-char EDRPOU length:  388


In [31]:
# We won't delete orgs from here because we do not have country filter

### Check the company codes and correct them

In [32]:
# check if we have empty company IDs or names
print('total records: ', len(companies_df))

empty_ids = companies_df.loc[(companies_df['company_id'] == '')] 
print('\nrecords with empty ids: ', len(empty_ids))
print('empty ids share: ', len(empty_ids)/len(companies_df))

empty_names = companies_df.loc[(companies_df['company_name'] == '')] 
print('\nrecords with empty names: ', len(empty_names))
print('empty names share: ', len(empty_names)/len(companies_df))


empty_names_ids = companies_df.loc[((companies_df['company_name'] == '')) & (companies_df['company_id'] == '')] 
print('\nrecords with both empty names and empty IDs: ', len(empty_names_ids))
print('both empty names and empty IDs share: ', len(empty_names_ids)/len(companies_df))


total records:  80203

records with empty ids:  6860
empty ids share:  0.08553296011371146

records with empty names:  6212
empty names share:  0.07745346184057952

records with both empty names and empty IDs:  6180
both empty names and empty IDs share:  0.07705447427153597


In [33]:
clean_companies_df = companies_df.loc[companies_df['company_id'] != ''].copy(deep=True)
len(clean_companies_df)

73343

In [34]:
print('non-ukraine companies', len(clean_companies_df[(clean_companies_df['company_ua'] != 1)]))
print('ukraine companies', len(clean_companies_df[(clean_companies_df['company_ua'] == 1)]))
print('ukraine companies with correct EDRPOU length', 
      len(clean_companies_df[(clean_companies_df['company_ua'] == 1) & (clean_companies_df['company_id'].str.len() == 8)]))
print('ukraine companies with too long EDRPOU length', 
      len(clean_companies_df[(clean_companies_df['company_ua'] == 1) & (clean_companies_df['company_id'].str.len() > 8)]))
print('ukraine companies with too short EDRPOU length', 
      len(clean_companies_df[(clean_companies_df['company_ua'] == 1) & (clean_companies_df['company_id'].str.len() < 8)]))

non-ukraine companies 945
ukraine companies 72398
ukraine companies with correct EDRPOU length 71077
ukraine companies with too long EDRPOU length 1319
ukraine companies with too short EDRPOU length 2


In [35]:
# Drop all entries where Ukraine's companu EDRPOU code is not correct (either too short or too long)
                        
clean_companies_df = clean_companies_df[((clean_companies_df['company_ua'] == 1) & 
                                           (clean_companies_df['company_id'].str.len() == 8)) | 
                                          (clean_companies_df['company_ua'] == 0)]


len(clean_companies_df)

72022

In [36]:
# Process edrpou code 
# Generally we delete all cases where length of EDRPOU code is < 4
# For Ukrainian companies - so that the length is 8 characters, if it is less, we add zeros before 


#p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 4)), 'company_id_r'] = '0000' + p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 4))]['company_id']
#p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 5)), 'company_id_r'] = '000' + p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 5))]['company_id']
#p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 6)), 'company_id_r'] = '00' + p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 6))]['company_id']
#p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 7)), 'company_id_r'] = '0' + p.loc[((p['company_ua'] == 1) & (p['company_id'].str.len() == 7))]['company_id']

#p[p['company_ua'] == 1]['company_id_r'].str.len().value_counts()

#p['company_id_r'].str.len().value_counts()

In [37]:
clean_companies_df.head(3)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,...,person,persons,company_ua,company_id,company_name,description,emitent_citizen,paid,full_name_office_position,unique_guids
99912,nacp_4a9ca6c4-8792-45f0-a09d-3a5767f2415e,2019-03-25 17:02:04,5120403.0,Щорічна,2019.0,Рогович Олександра Степанівна,Коломийська ЦРЛ,"Начальник відділу кадрів ,депутат Раківчицької...",,,...,,,1.0,37446383,"КЗ КРР ""КРЦ ПМСД""",Сестра медичної загальної практики сімейної ме...,"Юридична особа, зареєстрована в Україні",Оплачувана,рогович олександра степанівна коломийська црл ...,1
85754,nacp_af1d2e52-44f1-4980-88d4-d2c2645a307b,2019-03-27 15:39:58,1511346.0,Щорічна,2019.0,Дубинська Світлана Василівна,"Раківчицька сільськка рада, Будинок культури с...","Депутат сільської ради, директор Будинку культ...",,,...,,,1.0,02228411,Відділ культури Коломийської районної державно...,Керівник народного аматорського жіночого вокал...,"Юридична особа, зареєстрована в Україні",Оплачувана,дубинська світлана василівна раківчицька сільс...,1
98854,nacp_3938431d-8c8d-4d11-8f0e-106ca18d620d,2019-04-02 14:11:52,4864537.0,Щорічна,2019.0,Павленко Віктор Валерійович,"Ізмаїльська міська рада (ТОВ ""Стройсіті Ізмаїл"")",Депутат Ізмаїльської міської ради 7 скликання ...,,,...,,,1.0,26569293,Ізмаїльська міська рада,Депутат Ізмаїльської міської ради 7 скликання,"Юридична особа, зареєстрована в Україні",Не оплачувана,павленко віктор валерійович ізмаїльська міська...,1


### Create data sets of pairs,  company and org dicts

In [38]:
id_company_pairs = clean_companies_df.drop_duplicates(subset = ['id','company_id']) #['id','company_id','relation_type'])
print(len(clean_companies_df), len(id_company_pairs))

72022 72022


In [39]:
name_company_pairs = clean_companies_df.drop_duplicates(subset = ['full_name','company_id']) #['id','company_id','relation_type'])
print(len(clean_companies_df), len(name_company_pairs))

72022 71786


In [40]:
id_org_pairs = clean_organizations_df.drop_duplicates(subset = ['id','org_id']) #['id','company_id','relation_type'])
print(len(clean_organizations_df), len(id_org_pairs))

93699 93699


In [41]:
# Create a mapper for finding comapny name by company code
# Use the most popular comapny name 

company_map = clean_companies_df[['company_id', "company_name", "company_ua"]]
company_map = company_map.groupby(['company_id', "company_name", "company_ua"]).size().reset_index(name='counts')

comp_ids_and_counts = company_map.groupby(['company_id'])['counts'].agg('sum')

company_map = company_map.sort_values('counts', ascending=False).groupby('company_id').head(1)
company_map.drop('counts', axis=1,inplace=True)

company_map = company_map.join(comp_ids_and_counts, how='inner', on='company_id')

company_map

,company_id,company_name,company_ua,counts
14217,08751177,Національна академія внутрішніх справ,1.0,173
81,00013480,Міністерство фінансів України,1.0,158
19141,21560766,"ВАТ ""Укртелеком""",1.0,323
104,00017733,"АСК ""Укррічфлот""",1.0,359
6971,02070944,Київський національний університет імені Тарас...,1.0,90
...,...,...,...,...
22052,23810121,СФГ Прометей,1.0,1
22053,23811927,ПРИВАТНЕ ПІДПРИЄМСТВО ВІЯ,1.0,1
22054,23812284,приватно-науково виробниче підприємство Шератон,1.0,1
22055,23812893,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ КРЮКІВ...,1.0,2


In [42]:
# Create a mapper for finding organization name by company code
# Use the most popular organization name 

org_map = clean_organizations_df[['org_id', "org_name"]]
org_map = org_map.groupby(['org_id', "org_name"]).size().reset_index(name='counts')

org_ids_and_counts = org_map.groupby(['org_id'])['counts'].agg('sum')

org_map = org_map.sort_values('counts', ascending=False).groupby('org_id').head(1)
org_map.drop('counts', axis=1,inplace=True)

org_map = org_map.join(org_ids_and_counts, how='inner', on='org_id')

org_map

,org_id,org_name,counts
25932,38488439,Національна асоціація адвокатів України,2041
38572,42968570,Профспілкова організація персоналу Державної у...,1062
15597,35276302,УІФ ОПО Атестованих працівників ОВС,1089
39278,43357878,ГО СКП Прикарпаття,952
18748,36468689,Лікарняна каса столичної поліції,1579
...,...,...,...
14532,34950108,Первинна профспілкова організація Служби автом...,1
14531,34947912,Первинна профспілкова організація Запорізької ...,2
14529,34947708,Запорізька обласна Федерація шахів,1
14528,34944073,"""РУХ ЖИТТЯ""",1


### Save the data sets into .csv files

In [43]:
print("saving resulting data sets to csv files")

clean_companies_df.to_csv('company_02_clean_df.csv', index=False)
id_company_pairs.to_csv('company_02_id_pairs.csv', index=False)
company_map.to_csv('company_02_map.csv', index=False)


clean_organizations_df.to_csv('org_02_clean_df.csv', index=False)
id_org_pairs.to_csv('org_02_id_pairs.csv', index=False)
org_map.to_csv('org_02_map.csv', index=False)

saving resulting data sets to csv files


In [75]:
clean_companies_df.head(15)

,id,created_datetime,index,document_type,year,full_name,office,position,position_type,position_category,...,person,persons,company_ua,company_id,company_name,description,emitent_citizen,paid,full_name_office_position,unique_guids
99912,nacp_4a9ca6c4-8792-45f0-a09d-3a5767f2415e,2019-03-25 17:02:04,5120403.0,Щорічна,2019.0,Рогович Олександра Степанівна,Коломийська ЦРЛ,"Начальник відділу кадрів ,депутат Раківчицької...",,,...,,,1.0,37446383,"КЗ КРР ""КРЦ ПМСД""",Сестра медичної загальної практики сімейної ме...,"Юридична особа, зареєстрована в Україні",Оплачувана,рогович олександра степанівна коломийська црл ...,1
85754,nacp_af1d2e52-44f1-4980-88d4-d2c2645a307b,2019-03-27 15:39:58,1511346.0,Щорічна,2019.0,Дубинська Світлана Василівна,"Раківчицька сільськка рада, Будинок культури с...","Депутат сільської ради, директор Будинку культ...",,,...,,,1.0,02228411,Відділ культури Коломийської районної державно...,Керівник народного аматорського жіночого вокал...,"Юридична особа, зареєстрована в Україні",Оплачувана,дубинська світлана василівна раківчицька сільс...,1
98854,nacp_3938431d-8c8d-4d11-8f0e-106ca18d620d,2019-04-02 14:11:52,4864537.0,Щорічна,2019.0,Павленко Віктор Валерійович,"Ізмаїльська міська рада (ТОВ ""Стройсіті Ізмаїл"")",Депутат Ізмаїльської міської ради 7 скликання ...,,,...,,,1.0,26569293,Ізмаїльська міська рада,Депутат Ізмаїльської міської ради 7 скликання,"Юридична особа, зареєстрована в Україні",Не оплачувана,павленко віктор валерійович ізмаїльська міська...,1
31489,nacp_56095958-2f35-4067-a974-ccdf38546482,2019-04-07 12:57:45,3043677.0,Щорічна,2019.0,Мельник Юрій Володимирович,Хмельницька районна рада,Голова районної ради,Посада в органах місцевого самоврядування,третя категорія,...,1476687545344,,1.0,35344466,"Приватне підриємство ""Поділля Євро Будінвест-АС""",NaN,NaN,NaN,мельник юрій володимирович хмельницька районна...,1
88509,nacp_d27614d8-2e48-4c71-aaa5-388ef3da70bb,2019-07-17 16:39:52,2220230.0,Щорічна,2019.0,Іщенко Валентина Олексіївна,"ТОВ Домантівське, ПРАТ ЧОП Агротехсервіс",бухгалтер,,,...,,,1.0,03563548,ПРАТ ЧОП Агротехсервіс,бухгалтер,"Юридична особа, зареєстрована в Україні",Оплачувана,"іщенко валентина олексіївна тов домантівське, ...",1
5598,nacp_cc7d702f-a2b9-4321-b9f9-4158ece5d4dc,2019-08-07 17:14:17,1804809.0,Щорічна,2019.0,Шевчук Євген Антонович,"ТОВ ""АВ метал груп""","Менеджер по вопросам регионального развития, д...",,,...,1,,1.0,00381479,"ВАТ ""Дніпропетровський хлібокомбінат №2""",NaN,NaN,NaN,"шевчук євген антонович тов ""ав метал груп"" ме...",1
5050,nacp_7b8402e3-361c-4c4a-9b47-55f475d6291b,2019-12-31 18:57:57,1631469.0,Щорічна,2019.0,Нещимний Олександр Михайлович,"КП ШКЗ ""Імпульс""",Помічник начальника команди,Посадова особа юридичної особи публічного права,,...,1,,1.0,00191158,ПрАТ МК Азов сталь,NaN,NaN,NaN,"нещимний олександр михайлович кп шкз ""імпульс""...",1
99768,nacp_859ab1a3-228d-497a-8ae8-4739ef2c5741,2020-01-01 13:26:57,5084015.0,Щорічна,2019.0,Зубіцький Олександр Миколайович,Приватний підприємець,Приватний підприємець,,,...,,,1.0,04403025,Деражнянська міська рада,Депутат міської ради,"Юридична особа, зареєстрована в Україні",Не оплачувана,зубіцький олександр миколайович приватний підп...,1
6222,nacp_d5c9dfa6-2718-4eb1-90da-bf0834f8fc06,2020-01-01 15:00:11,1978076.0,Щорічна,2019.0,Дейко Петро Григорович,ТОВ Теплоенергоцентр Роганського промвузла,Головний метролог,,,...,1,,1.0,14085922,"ПрАТ ""Теплоенергетичний Центр Роганського пром...",NaN,NaN,NaN,дейко петро григорович тов теплоенергоцентр ро...,1
28114,nacp_d5c9dfa6-2718-4eb1-90da-bf0834f8fc06,2020-01-01 15:00:11,1978076.0,Щорічна,2019.0,Дейко Петро Григорович,ТОВ Теплоенергоцентр Роганського промвузла,Головний метролог,,,...,1,,1.0,42509827,"ТОВ ""ТЕПЛОЕНЕРГОЦЕНТР РОГАНСЬКОГО ПРОМВУЗЛА""",NaN,NaN,NaN,дейко петро григорович тов теплоенергоцентр ро...,1


In [72]:
clean_unique_declarations_with_connection = set(list(clean_organizations_df.id) + list(clean_companies_df.id))
print(len(clean_unique_declarations_with_connection))

clean_unique_declarations_with_connection = set(list(clean_organizations_df.full_name_office_position) + list(clean_companies_df.full_name_office_position))
print(len(clean_unique_declarations_with_connection))

print(len(clean_organizations_df), len(clean_companies_df))

113393
113336
93699 72022


In [53]:
clean_companies_df.columns
#head(3)

Index(['id', 'created_datetime', 'index', 'document_type', 'year', 'full_name',
       'office', 'position', 'position_type', 'position_category',
       'corruptionAffected', 'dnt_organization_group', 'city', 'city_type',
       'relation_type', 'person', 'persons', 'company_ua', 'company_id',
       'company_name', 'description', 'emitent_citizen', 'paid',
       'full_name_office_position', 'unique_guids'],
      dtype='object')

In [51]:
clean_companies_df.groupby('office')[['id']].count().sort_values(['id'],ascending=False).head(15)

,id
office,
Верховна Рада України,1421
Національний банк України,285
Київська міська рада,278
Верховний Суд,192
Приватний підприємець,182
Харківська обласна рада,159
Харківська міська рада,151
Збройні Сили України,150
Апарат Верховної Ради України,137


In [54]:
clean_companies_df.groupby('dnt_organization_group')[['id']].count().sort_values(['id'],ascending=False).head(15)

,id
dnt_organization_group,
Місцеві адміністрації та ради,27227
Без категорії,23638
"Кабмін, міністерства та підлеглі органи",10875
Суд,3100
"Інші державні служби, комісії, і т.п.",2296
Парламент,1988
Прокуратура,897
Пенсійний фонд,840
НБУ,338


In [56]:
clean_companies_df.groupby('dnt_organization_group')[['id']].nunique().sort_values(['id'],ascending=False).head(15)

,id
dnt_organization_group,
Місцеві адміністрації та ради,17850
Без категорії,15794
"Кабмін, міністерства та підлеглі органи",8518
Суд,2149
"Інші державні служби, комісії, і т.п.",1787
Прокуратура,679
Пенсійний фонд,673
Парламент,427
НБУ,240
